In [90]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import requests
import json
import arrow

In [3]:
backpack_tf_api_key = "568d36d7ba8d889e3ba94bdf"

In [413]:
class Item:
    
    history = DataFrame()
    
    def __repr__(self):
        return str(self.history)

    def __init__(self, item=None, **kwargs):
        if item:
            # Make the API call.
            params = {"key": backpack_tf_api_key, "item": item}
            params.update(kwargs)
            data = json.loads(requests.get("http://backpack.tf/api/IGetPriceHistory/v1/",
                                           params=params).text)['response']['history']
            # Parse it into a `pandas` `DataFrame`.
            frame = DataFrame(data,
                              index=[np.datetime64(str(arrow.get(t['timestamp']).format('YYYY-MM-DD'))) for t in data],
                              columns=["currency", "value", "value_high"])
            # Throw out earlier dates, for the moment. `2013-01-01` is the starting point for all knowledge.
            frame = frame[pd.to_datetime('2013-01-01'):]
            # Extrapolate Refined Metal price back to `2013-01-01`.
            if item == "Refined Metal":
                frame.loc[pd.to_datetime('2013-01-01')] = ['usd', 0.40, 0.40]
                frame = frame.sort_index()
            # Rebase rows from `2013-01-01` forward to USD.
            else:
                for i in range(0, len(frame.index)):
                    data = frame.iloc[i]
                    date = data.name
                    if date >= pd.to_datetime('2013-01-01'):
                        currency = data['currency']
                        conversion_rate = value_at(currency, date)
                        frame.iat[i, 0] = 'usd'
                        frame.iat[i, 1] *= conversion_rate[0]
                        frame.iat[i, 2] *= conversion_rate[1]
            self.history = frame

            
    def to_csv(self, filename):
        self.history.to_csv(filename)
    
    
    def read_csv(self, filename):
        self.history = self.history.from_csv(filename)

        
def value_at(currency, date):
    f_date = np.datetime64(str(arrow.get(date).format('YYYY-MM-DD')))
    # USD-to-USD conversion is a multiple of 1.
    if currency == "usd":
        return (1, 1)
    # Convert metal prices based on historical data.
    elif currency == "metal":
        value_entry = metal.history[:f_date].tail(1)
        return (value_entry['value'][0], value_entry['value_high'][0])
    # Convert key prices based on historical data.
    elif currency == "keys":
        value_entry = key.history[:f_date].tail(1)
        metal_conversion = value_at("metal", date)
        return (metal_conversion[0] * value_entry['value'][0],
                metal_conversion[1] * value_entry['value_high'][0])

`backpack.tf` data goes back to late 2012: the initial log for Earbuds goes back to `2012-08-10`, while the initial log for keys goes back to `2012-07-09`. The constraining variable is Metal, the USD price for which was only first logged on `2013-01-07`. Since all prices are passed through Metal on the way to USD, this means that reliable prices can only be established back to (a reasonably extrapolated) `2013-01-01`.

Future work may extend the augment the timeline back beyond this date, this is up in the air - still exploring alternative data sources.

For the moment price data older than `2013-01-01` is removed. This could change in the final version.

In [415]:
metal = Item("Refined Metal")
keys = Item("Mann Co. Supply Crate Key")
earbuds = Item("Earbuds")

In [385]:
metal

           currency  value  value_high
2013-01-01      usd   0.40        0.40
2013-01-20      usd   0.40        0.40
2013-02-03      usd   0.38        0.40
2013-02-19      usd   0.36        0.38
2013-03-04      usd   0.35        0.37
2013-03-16      usd   0.35        0.35
2013-06-21      usd   0.33        0.35
2013-08-24      usd   0.31        0.33
2013-09-15      usd   0.30        0.32
2013-10-13      usd   0.30        0.31
2013-10-24      usd   0.30        0.32
2013-11-17      usd   0.31        0.32
2013-12-02      usd   0.30        0.32
2014-01-09      usd   0.29        0.30
2014-02-17      usd   0.26        0.28
2014-03-29      usd   0.25        0.26
2014-04-27      usd   0.25        0.25
2014-05-17      usd   0.24        0.25
2014-10-22      usd   0.21        0.23
2014-11-13      usd   0.19        0.21
2014-12-05      usd   0.17        0.18
2014-12-14      usd   0.16        0.17
2014-12-26      usd   0.16        0.16
2015-01-22      usd   0.15        0.15
2015-02-11      usd   0.1

In [386]:
earbuds

           currency     value  value_high
2013-01-01      usd  32.34400    34.63200
2013-01-07      usd  33.48800    35.96400
2013-01-30      usd  35.96400    39.52800
2013-02-04      usd  35.29440    40.99200
2013-02-26      usd  35.57520    39.80880
2013-03-02      usd  35.28720    39.80880
2013-03-12      usd  36.66600    41.05890
2013-03-18      usd  37.80000    40.27800
2013-03-27      usd  38.83950    39.87900
2013-03-30      usd  38.40200    40.91850
2013-04-03      usd  38.40200    39.40300
2013-04-10      usd  37.88750    40.40400
2013-04-11      usd  37.88750    39.81250
2013-04-12      usd  36.37200    39.81250
2013-04-13      usd  37.29600    39.14400
2013-04-15      usd  35.74200    39.14400
2013-04-16      usd  36.62750    37.51300
2013-04-20      usd  35.03500    37.51300
2013-04-22      usd  35.88200    36.72900
2013-04-27      usd  35.05950    38.50000
2013-05-13      usd  38.50000    39.34700
2013-05-19      usd  38.50000    42.02100
2013-05-21      usd  40.25000    4